
# Step  #2: Data Cleaning
## Taking care of missing entries or columns with no numeric data

## About this step
In this step,  I'll look at the data, detect, and deal with it while concentrating on:

* Data that is duplicated
* Data that has gone missing
* Casting of data types

### import modules (packages)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline

### Load the Data of Terrorism dataset

In [2]:
df = pd.read_csv('data_of_terrorism.csv')
df.drop('Unnamed: 0', inplace=True, axis=1)



In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 30 columns):
CITY                                         30000 non-null object
COUNTRY                                      30000 non-null object
Claimed Responsibility                       30000 non-null object
DATE                                         30000 non-null object
FATALITIES                                   30000 non-null object
Group Name of Perpetrator Group              30000 non-null object
Group Sub Name of Perpetrator Group          30000 non-null object
Hostages                                     30000 non-null object
INJURED                                      30000 non-null object
LOCATION DETAILS                             30000 non-null object
Name of Entity                               30000 non-null object
Nationality of Target                        30000 non-null object
Number of Perpetrator Fatalities             30000 non-null object
Number of Perpetrato

#### Now I'll convert the 'Unknown' values to np.nan 

In [4]:
df = df.replace('Unknown', np.nan)

    You can now identify which columns have data missing.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 30 columns):
CITY                                         28173 non-null object
COUNTRY                                      30000 non-null object
Claimed Responsibility                       30000 non-null object
DATE                                         30000 non-null object
FATALITIES                                   27985 non-null object
Group Name of Perpetrator Group              15252 non-null object
Group Sub Name of Perpetrator Group          1261 non-null object
Hostages                                     29963 non-null object
INJURED                                      27017 non-null object
LOCATION DETAILS                             13435 non-null object
Name of Entity                               23581 non-null object
Nationality of Target                        29847 non-null object
Number of Perpetrator Fatalities             29275 non-null object
Number of Perpetrator

#### I'll now see whether there are any duplicate rows, and if there are, I'll remove them.

In [6]:
print("Inside the data table there are", df.duplicated().sum() ,"duplicates")

Inside the data table there are 1386 duplicates


In [7]:
df = df.drop_duplicates().reset_index(drop=True).copy()

In [8]:
df.shape

(28614, 30)

### Check the amount of missing values in each column

In [9]:
for c in df.columns:
    missing = df[c].isna().sum()
    print("in column {} there are {:.3f}% missing values".format(c, missing*100/len(df)))

in column CITY there are 6.060% missing values
in column COUNTRY there are 0.000% missing values
in column Claimed Responsibility there are 0.000% missing values
in column DATE there are 0.000% missing values
in column FATALITIES there are 6.790% missing values
in column Group Name of Perpetrator Group there are 48.948% missing values
in column Group Sub Name of Perpetrator Group there are 95.645% missing values
in column Hostages there are 0.101% missing values
in column INJURED there are 10.128% missing values
in column LOCATION DETAILS there are 54.033% missing values
in column Name of Entity there are 20.619% missing values
in column Nationality of Target there are 0.531% missing values
in column Number of Perpetrator Fatalities there are 2.523% missing values
in column Number of Perpetrators there are 87.471% missing values
in column PERPETRATOR GROUP there are 48.948% missing values
in column PROVINCE/ADMINISTRATIVE REGION/U.S. STATE there are 0.706% missing values
in column Prop

### Create dictionary of terrorist cities

    To complete the CITY column, I'll utilize the COUNTRY column. First, I'll construct a dictionary that will provide each nation a city where a terrorist incident occurred. Then, with the use of this dictionary, I'll fill in the missing cities.

In [10]:
Terrorist_cities = dict()
for row in range(df.shape[0]):
    if (type(df.CITY[row]) ==  type("string")):                                    #   df.CITY[row] != np.nan
        Terrorist_cities.update({df.COUNTRY[row] : df.CITY[row]})
Terrorist_cities
        


{'Algeria': 'Bordj Menaiel',
 'Bahrain': 'Demistan',
 'Egypt': 'Arish',
 'Iran': 'Zabol',
 'Iraq': 'Mosul',
 'Israel': 'Afula',
 'Jordan': 'Maan',
 'Kuwait': 'Kuwait',
 'Lebanon': 'Jlala',
 'Libya': 'Benghazi',
 'Morocco': 'Casablanca',
 'Qatar': 'Doha',
 'Saudi Arabia': 'Hafr al-Batin',
 'Syria': 'Damascus',
 'Tunisia': 'Sousse',
 'Turkey': 'Cizre',
 'United Arab Emirates': 'Abu Dhabi',
 'West Bank and Gaza Strip': 'Huwwara',
 'Western Sahara': 'Tifariti',
 'Yemen': 'Ahwar'}

In [11]:
for row in range(df.shape[0]):
    if (type(df.CITY[row]) !=  type("string")):                                       #   df.CITY[row] == np.nan
        df.CITY[row] = Terrorist_cities[df.COUNTRY[row]]


    Columns with a significant absence of data will be removed from the table.

In [12]:
df =df.drop(columns=['Total Number of Fatalities','Total Number of Injured','Number of Perpetrators',"Group Sub Name of Perpetrator Group"])


    I'm having to create a dictionary to convert columns with only "yes" and "no" values to binary values.

In [13]:
replace_map = {'No':0, 'Yes':1}

    Now I'll use the dictionary I created to convert data in columns containing "yes" and "no" values to binary values, 
    and then fill in the blanks with the most common value.

In [14]:
df.Hostages.replace(replace_map,inplace = True)

In [15]:
df.Hostages = df.Hostages.fillna(df.Hostages.mode()[0])

In [16]:
df.Ransom.replace(replace_map,inplace = True)

In [17]:
df.Ransom = df.Ransom.fillna(df.Ransom.mode()[0])

In [18]:
df["Property Damage"].replace(replace_map,inplace = True)

In [19]:
df["Property Damage"] = df["Property Damage"].fillna(df["Property Damage"].mode()[0])

In [20]:
df["Successful Attack?"].replace(replace_map,inplace = True)

In [21]:
df["Suicide_Attack"].replace(replace_map,inplace = True)

    I'll use the median number to fill in the blanks in columns with continuous values.

In [22]:
df.FATALITIES = df.FATALITIES.fillna(df.FATALITIES.median())

In [23]:
df.INJURED  = df.INJURED.fillna(df.INJURED.median())

In [24]:
df['Number of Perpetrator Fatalities'] = df['Number of Perpetrator Fatalities'].fillna(df['Number of Perpetrator Fatalities'].median())

    Now I'll double-check that all of the numbers in the column are integers.

In [25]:
 df['FATALITIES'] = df['FATALITIES'].astype(int)

In [26]:
 df['INJURED'] = df['INJURED'].astype(int)

In [27]:
 df['Number of Perpetrator Fatalities'] = df['Number of Perpetrator Fatalities'].astype(int)

    I'll fill in the missing data in the most common value in columns that include strings.

In [28]:
df["TARGET TYPE"] = df["TARGET TYPE"].fillna(df["TARGET TYPE"].mode()[0])

In [29]:
df["Specific Description"] = df["Specific Description"].fillna(df["Specific Description"].mode()[0])

In [30]:
df["Name of Entity"] = df["Name of Entity"].fillna(df["Name of Entity"].mode()[0])

In [31]:
df["Type of Attack"] = df["Type of Attack"].fillna(df["Type of Attack"].mode()[0])

In [32]:
df["Weapon Sub_type"] = df["Weapon Sub_type"].fillna(df["Weapon Sub_type"].mode()[0])

In [33]:
df["Weapon Type"] = df["Weapon Type"].fillna(df["Weapon Type"].mode()[0])

    I'll fill in the missing data in the Objectives column in the nation where the terrorist attack occurred.

In [34]:
for row in range(df.shape[0]):
    if ( type(df["Nationality of Target"][row]) != type("string")):                                       #   df["Nationality of Target"][row] == np.nan
        df["Nationality of Target"][row] = df.COUNTRY[row]


    I'll generate a unique dictionary for each column that contains strings, converting each string to a unique numeric value.

In [35]:
replace_of_Claimed_Responsibility = {'No':0,  'Yes (Confirmed: Unknown; Mode: Video)':1 ,'Yes (Confirmed: Unknown; Mode: E-mail)':2, 'Yes (Confirmed: Unknown; Mode: Posted to website, blog, etc.)':3, 'Yes (Confirmed: Unknown; Mode: Unknown)':4, 'Yes (Confirmed: Unknown; Mode: Personal claim)':5, 'Yes (Confirmed: Unknown; Mode: Note left at scene)':6, 'Yes (Confirmed: Unknown; Mode: Call (pre-incident))':7, 'Yes (Confirmed: Unknown; Mode: Call (post-incident))':8, 'Yes (Confirmed: Unknown; Mode: Other)':9}

In [36]:
df["Claimed Responsibility"].replace(replace_of_Claimed_Responsibility,inplace = True)

In [37]:
for row in range(df.shape[0]):
    if ( type(df["PROVINCE/ADMINISTRATIVE REGION/U.S. STATE"][row]) != type("string")):                                       #   df["Nationality of Target"][row] == np.nan
        df["PROVINCE/ADMINISTRATIVE REGION/U.S. STATE"][row] = df.COUNTRY[row]


In [38]:
dict_of_Type_of_Attack = dict()
for i in range(len(df['Type of Attack'].unique())):
    dict_of_Type_of_Attack[df['Type of Attack'].unique()[i]] = i
    print(df['Type of Attack'].unique()[i], " == ", i)


Bombing/Explosion  ==  0
Facility/Infrastructure Attack  ==  1
Armed Assault  ==  2
Hostage Taking (Kidnapping)  ==  3
Assassination  ==  4
Hostage Taking (Barricade Incident)  ==  5
Hijacking  ==  6
Unarmed Assault  ==  7


In [39]:
df["Type of Attack"].replace(dict_of_Type_of_Attack,inplace = True)

In [40]:
dict_of_Weapon_Sub_type = dict()
for i in range(len(df['Weapon Sub_type'].unique())):
    dict_of_Weapon_Sub_type[df['Weapon Sub_type'].unique()[i]] = i
    print(df['Weapon Sub_type'].unique()[i], " == ", i)



Unknown Explosive Type  ==  0
Automatic Weapon  ==  1
Unknown Gun Type  ==  2
Projectile (rockets, mortars, RPGs, etc.)  ==  3
Vehicle  ==  4
Sticky Bomb  ==  5
Land Mine  ==  6
Remote Trigger  ==  7
Rifle/Shotgun (non-automatic)  ==  8
Suicide (carried bodily by human being)  ==  9
Molotov Cocktail/Petrol Bomb  ==  10
Knife or Other Sharp Object  ==  11
Unknown Weapon Type  ==  12
Arson/Fire  ==  13
Grenade  ==  14
Handgun  ==  15
Other Explosive Type  ==  16
Pipe Bomb  ==  17
Blunt Object  ==  18
Dynamite/TNT  ==  19
Rope or Other Strangling Device  ==  20
Time Fuse  ==  21
Hands, Feet, Fists  ==  22
Explosive  ==  23
Poisoning  ==  24
Pressure Trigger  ==  25
Suffocation  ==  26
Gasoline or Alcohol  ==  27
Other Gun Type  ==  28
Letter Bomb  ==  29


In [41]:
df["Weapon Sub_type"].replace(dict_of_Weapon_Sub_type,inplace = True)


In [42]:
dict_of_Weapon_Type = dict()
for i in range(len(df['Weapon Type'].unique())):
    dict_of_Weapon_Type[df['Weapon Type'].unique()[i]] = i
    print(df['Weapon Type'].unique()[i], " == ", i)

Explosives/Bombs/Dynamite  ==  0
Firearms  ==  1
Incendiary  ==  2
Melee  ==  3
Chemical  ==  4
Sabotage Equipment  ==  5
Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)  ==  6
Other  ==  7
Biological  ==  8


In [43]:
df["Weapon Type"].replace(dict_of_Weapon_Type,inplace = True)

In [44]:
dict_of_COUNTRY = dict()
for i in range(len(df['COUNTRY'].unique())):
    dict_of_COUNTRY[df['COUNTRY'].unique()[i]] = i
    print(df['COUNTRY'].unique()[i], " == ", i)                    

Iraq  ==  0
Syria  ==  1
Yemen  ==  2
Egypt  ==  3
Saudi Arabia  ==  4
Israel  ==  5
Libya  ==  6
West Bank and Gaza Strip  ==  7
Turkey  ==  8
Lebanon  ==  9
Jordan  ==  10
Tunisia  ==  11
Iran  ==  12
Morocco  ==  13
Algeria  ==  14
Western Sahara  ==  15
Bahrain  ==  16
Kuwait  ==  17
Qatar  ==  18
United Arab Emirates  ==  19


In [45]:
df["COUNTRY"].replace(dict_of_COUNTRY,inplace = True)

    I used the datetime library to convert each date into a unique numeric value for the dates column.

In [46]:
for row in range(df.shape[0]):
    date_time_str = df["DATE"][row]
    date = datetime.strptime(date_time_str[2:], '%y-%m-%d')
    df["DATE"][row] = datetime.timestamp(date)
    


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28614 entries, 0 to 28613
Data columns (total 25 columns):
CITY                                         28614 non-null object
COUNTRY                                      28614 non-null int64
Claimed Responsibility                       28614 non-null int64
DATE                                         28614 non-null float64
FATALITIES                                   28614 non-null int64
Group Name of Perpetrator Group              14608 non-null object
Hostages                                     28614 non-null float64
INJURED                                      28614 non-null int64
LOCATION DETAILS                             13153 non-null object
Name of Entity                               28614 non-null object
Nationality of Target                        28614 non-null object
Number of Perpetrator Fatalities             28614 non-null int64
PERPETRATOR GROUP                            14608 non-null object
PROVINCE/ADMINISTRATIVE

In [48]:
df.describe(include='all')

,CITY,COUNTRY,Claimed Responsibility,DATE,FATALITIES,Group Name of Perpetrator Group,Hostages,INJURED,LOCATION DETAILS,Name of Entity,...,Ransom,Specific Description,Successful Attack?,Suicide_Attack,TARGET TYPE,Type of Attack,Weapon Sub_type,Weapon Type,latitude,longitude
count,28614,28614.000000,28614.000000,2.861400e+04,28614.000000,14608,28614.000000,28614.000000,13153,28614,...,28614.000000,28614,28614.000000,28614.000000,28614,28614.00000,28614.000000,28614.000000,28614.000000,28614.000000
unique,4520,NaN,NaN,NaN,NaN,307,NaN,NaN,8692,3949,...,NaN,6115,NaN,NaN,244,NaN,NaN,NaN,NaN,NaN
top,Baghdad,NaN,NaN,NaN,NaN,Islamic State of Iraq and the Levant (ISIL),NaN,NaN,The incident occurred in the Balad Ruz district.,Not Applicable,...,NaN,Civilians,NaN,NaN,Private Citizens & Property,NaN,NaN,NaN,NaN,NaN
freq,3703,NaN,NaN,NaN,NaN,5885,NaN,NaN,55,12719,...,NaN,5367,NaN,NaN,11176,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.031628,0.627036,1.459446e+09,2.820927,NaN,0.079262,3.373454,NaN,NaN,...,0.002586,NaN,0.843573,0.082792,NaN,0.79503,2.949151,0.307227,30.782069,39.815550
std,NaN,2.862513,1.453982,5.084451e+07,15.327379,NaN,0.270152,12.999723,NaN,NaN,...,0.050789,NaN,0.363266,0.275572,NaN,1.33175,3.997059,0.707988,7.064758,8.798183
min,NaN,0.000000,0.000000,1.381784e+09,0.000000,NaN,0.000000,0.000000,NaN,NaN,...,0.000000,NaN,0.000000,0.000000,NaN,0.00000,0.000000,0.000000,-28.725386,-84.284113
25%,NaN,0.000000,0.000000,1.416024e+09,0.000000,NaN,0.000000,0.000000,NaN,NaN,...,0.000000,NaN,1.000000,0.000000,NaN,0.00000,0.000000,0.000000,31.444729,36.254771
50%,NaN,0.000000,0.000000,1.454105e+09,1.000000,NaN,0.000000,1.000000,NaN,NaN,...,0.000000,NaN,1.000000,0.000000,NaN,0.00000,2.000000,0.000000,33.302431,43.644167
75%,NaN,3.000000,0.000000,1.495228e+09,2.000000,NaN,0.000000,4.000000,NaN,NaN,...,0.000000,NaN,1.000000,0.000000,NaN,2.00000,4.000000,0.000000,34.186303,44.378799


    I'm going to save the new table in CSV format right now.

In [49]:
df.to_csv('data_of_terrorism_after_cleaning.csv')
